# Backend: FastAPI

An API (Application Programming Interface) is just a small program that answers requests. You send it a question like “give me item 42,” it sends back an answer in JSON. 

Think of it as a helpful librarian for your data.

**Note on JSON**: JSON (JavaScript Object Notation) is a lightweight, text-based format for structuring data as key–value pairs and arrays that’s easy for humans to read and for machines to parse.


[FastAPI](https://github.com/fastapi/fastapi) publishes a **menu** of everything your API serves (paths, inputs, outputs).


Generally, you would open `http://127.0.0.1:8000/docs` and you get buttons you can click to try the API. That “menu” (called **OpenAPI**) is also usable by other tools to interact with your app.


In **full stack**, FastAPI is typically the **backend**: it runs on the server, exposes HTTP APIs, handles business logic, talks to databases/queues, and is served by an ASGI server like **Uvicorn**; while the frontend (e.g., Next.js/[React](https://en.wikipedia.org/wiki/React_(software))) calls those FastAPI endpoints over HTTP.


**ASGI** (Asynchronous Server Gateway Interface) is the Python standard that connects **web servers** to **async-capable apps** (like FastAPI), enabling concurrency and long-lived connections (i.e., can handle multiple requests in overlapping time). 

In [1]:
!pip install "fastapi>=0.115" "uvicorn[standard]>=0.30"

  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached pydantic-2.12.3-py3-none-any.whl.metadata (87 kB)
  Using cached annotated_doc-0.0.3-py3-none-any.whl.metadata (6.6 kB)
  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached httptools-0.7.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.5 kB)
  Using cached uvloop-0.22.1-cp312-cp312-macosx_10_13_universal2.whl.metadata (4.9 kB)
  Using cached watchfiles-1.1.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.9 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.41.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.3 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
Using cached uvicorn-0.38.0-py3-none-any.whl (68 kB)
Using cached annotated_doc-0.0.3-py3-none-any.whl (5.5 kB)
Using cached click-8.3.0-py3-none-any.whl (107 kB)
Using cached httptools-0.7.1-cp312-cp312-macosx_11_0_arm64.whl (110 kB)
Using cached pyd

To test a simple example, let us create a `main.py` file.

```python
# main.py
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Item(BaseModel):
    name: str
    price: float

@app.get("/")
def hello():
    return {"message": "Hello 👋"}

@app.post("/items")
def create(item: Item):
    return {"ok": True, "item": item}
```

You can then run the app with:

```bash
uvicorn main:app --reload
```

A common error that main appear is: 
```
ERROR:    [Errno 48] Address already in use
```

You can see which ports are busy with 

```
lsof -iTCP -sTCP:LISTEN -n -P
```

Or if a specific port is busy with:
```
lsof -i :8000
```

Then visit [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs).

click `POST /items`, try sending `{"name": 123, "price": "oops"}` and see it politely refuse with a helpful error.

Then send `{"name":"apple","price":1.2}` and it works.

You can also `curl` to your app:

In [6]:
!curl -X 'POST' \
  'http://127.0.0.1:8000/items' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{"name":"apple","price":1.2}'

{"ok":true,"item":{"name":"apple","price":1.2}}

In [4]:
!curl -X 'POST' \
  'http://127.0.0.1:8000/items' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{"name":"apple","price":"p"}'

{"detail":[{"type":"float_parsing","loc":["body","price"],"msg":"Input should be a valid number, unable to parse string as a number","input":"p"}]}

In [4]:
!curl http://127.0.0.1:8000/

{"message":"Hello 👋"}

In these `curl` commands the `-X` option is setting teh HTTP method. You usually don’t need `-X` for `GET` as curl defaults to GET if no body is sent.

Let us take a closer look at the program.

### Declaration

The first two lines simply import the dependencies:


- `FastAPI` is the web framework class you’ll instantiate.

- `BaseModel` (from [Pydantic](https://docs.pydantic.dev/latest/)) is used to define data shapes with type hints and validation.


Then,

```python
app = FastAPI()
```

creates the app. `app` is the ASGI application, run/served by Uvicorn.

Then, 

### Shape of data

```python
class Item(BaseModel):
    name: str
    price: float
```

Defines the shape of data your API expects for an `Item`. `BaseModel` makes it a Pydantic model:
 - Validates types (e.g., name must be text, price must be a number).
 - Converts compatible types (e.g., "1.2" → 1.2) and raises a 422 Unprocessable Entity if invalid.


### Endpoint

`@app.get("/")` declares a GET endpoint at the root path `/`

```python
@app.get("/")
def hello():
    return {"message": "Hello 👋"}
```

When a GET request hits /, FastAPI calls hello() and serializes the returned Python dict to JSON:

- Response body: {"message": "Hello 👋"}
- Default status code: `200 OK`


`@app.post("/items")` declares a POST endpoint at /items.

```python
@app.post("/items")
def create(item: Item):
    return {"ok": True, "item": item}
```

The parameter `item: Item` tells FastAPI:

*Read the request body as JSON, parse & validate it against the Item model.*

If the body is missing/invalid → return 422 with error details.

If valid, you get a Python Item instance inside the function.

Returning {"ok": True, "item": item}:

FastAPI sees item is a Pydantic model, converts it to JSON automatically.

Default status code is `200 OK`


### Port


* **`http://`** — the protocol (how we talk). Here it is plain [HTTP](https://en.wikipedia.org/wiki/HTTP).
  
* **`127.0.0.1`** — the **loopback** address, aka **localhost**. It always means “this computer”. Only apps on the same machine can reach it.


* **`:8000`** — the **port** number (like a door on the house). Uvicorn’s dev server defaults to 8000 unless you change it.

  
* (No extra path) **`/`** — the **root** of your API. The “root path” / is a URL path, **not a filesystem path**. It just means “the endpoint at the top of the API” (e.g., GET /).

So `http://127.0.0.1:8000` = “Use HTTP to talk to the API running on *my* machine, on port 8000, at the root path.”

### Handy extras

* You can also use: `http://localhost:8000/`
* The docs live at: `http://127.0.0.1:8000/docs` 
* Change port/host when running Uvicorn:

  ```bash
  # different port
  uvicorn main:app --reload --port 3000

  # allow other devices on your network to access it (dev only!)
  uvicorn main:app  --host 127.0.0.1 --port 8000
  ```

* Print just body with `curl`

```bash
curl -s http://127.0.0.1:8000/ | jq
```

The `-s` option silences the progress bars.


## Adding Endpoint


In this example we add an FFT endpoint

```python
@app.get("/fft")
def fft_endpoint(n: int = 4096, L: float = 2*np.pi, k: int = 0):
    """
    Compute FFT of sin(5x)*cos(9x) on [0,L) with N samples and return the value at harmonic k.
    - n: number of samples
    - L: domain length
    - k: harmonic index (e.g., 0, ±4, ±14). Maps to FFT bin m ≈ k*L/(2π).
    """
    try:
        return fft_at_k(k=k, n=n, L=L)
    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))
```

where `fft_at_k` is defined somewhere else (maybe in the same file, or in another package).


### Endpoint types

Here `/fft` endpoint is a pure read-only calculation: given inputs (k, n, L) it returns a result and doesn’t create/modify/delete server state. 
It is therefore a `GET` request. 


Other endpoints include `POST`, `PUT`, `DELETE`. Here is a brief summary of what these do:


* **GET** — *Read-only, safe, idempotent, cacheable.*
  Use for fetching or computing without changing server state.

  ```bash
  curl -s -X GET "http://127.0.0.1:8000/fft?k=14&n=4096&L=6.283185307179586"
  ```

  Typical codes: `200 OK`, `304 Not Modified`, `404 Not Found`.

* **POST** — *Create or submit; not idempotent.*
  Use for creating resources, submitting forms/jobs, or sending complex inputs in the body.

  ```bash
  curl -s -X POST http://127.0.0.1:8000/items \
    -H "Content-Type: application/json" \
    -d '{"name":"notebook","price":12.5}'
  ```

  Typical codes: `201 Created`, `202 Accepted`, `400/422` validation errors.

* **PUT** — *Replace (or create at known URL); idempotent.*
  Use to fully replace a resource. Same request repeated → same result.

  ```bash
  curl -s -X PUT http://127.0.0.1:8000/items/42 \
    -H "Content-Type: application/json" \
    -d '{"name":"notebook","price":13.0}'
  ```

  Typical codes: `200 OK`, `201 Created`, `204 No Content`.

* **PATCH** — *Partial update; not necessarily idempotent (often treated as such).*
  Use to modify part of a resource.

  ```bash
  curl -s -X PATCH http://127.0.0.1:8000/items/42 \
    -H "Content-Type: application/json" \
    -d '{"price":13.5}'
  ```

  Typical codes: `200 OK`, `204 No Content`.

* **DELETE** — *Remove; idempotent by convention.*
  Repeating a successful DELETE keeps it gone.

  ```bash
  curl -s -X DELETE http://127.0.0.1:8000/items/42
  ```

  Typical codes: `200 OK`, `202 Accepted`, `204 No Content`, `404 Not Found`.

* **HEAD** — *Headers only (like GET without body).* Useful for checks.

  ```bash
  curl -I http://127.0.0.1:8000/fft?k=14
  ```

* **OPTIONS** — *What methods are allowed / CORS preflight.*

  ```bash
  curl -s -X OPTIONS http://127.0.0.1:8000/fft -i
  ```

**Rules of thumb**

* Read-only? → **GET**.
* Create/submit/long payloads? → **POST**.
* Full replace at known URL? → **PUT**.
* Partial update? → **PATCH**.
* Delete? → **DELETE**.

(And for APIs: document expected status codes and request/response schemas.)

In this context, **Idempotent** means that doing the same request multiple times has the same effect as doing it once.


## Status codes


Sending HTTP requests usually ends with status codes being received. 

**1xx — Informational**

* **100 Continue** – Client may send the body.
* **101 Switching Protocols** – Upgrading (e.g., to WebSocket).
* **102 Processing** – Server has accepted but not finished (WebDAV).

**2xx — Success**

* **200 OK** – Standard successful response (GET/PUT/PATCH/DELETE).
* **201 Created** – New resource created (usually with `Location` header).
* **202 Accepted** – Accepted for async processing (job queued).
* **204 No Content** – Success, no response body (e.g., DELETE or PUT with no body).
* **206 Partial Content** – Ranged responses.

**3xx — Redirection**

* **301 Moved Permanently** – Resource has a new URL.
* **302 Found** – Temporary redirect (common but legacy semantics).
* **303 See Other** – Redirect for POST/redirect/GET pattern.
* **304 Not Modified** – Use cached copy (conditional GET).
* **307 Temporary Redirect** – Redirect, **preserve method**.
* **308 Permanent Redirect** – Permanent, **preserve method**.

**4xx — Client Errors**

* **400 Bad Request** – Malformed syntax/params.
* **401 Unauthorized** – Missing/invalid auth (use with `WWW-Authenticate`).
* **403 Forbidden** – Authenticated but not allowed.
* **404 Not Found** – No such resource/route.
* **405 Method Not Allowed** – Wrong HTTP method for this route.
* **406 Not Acceptable** – Content negotiation failed (e.g., `Accept` header).
* **408 Request Timeout** – Client took too long to send.
* **409 Conflict** – Version/edit conflict, duplicate, business rule clash.
* **410 Gone** – Resource intentionally removed.
* **412 Precondition Failed** – ETag/time precondition failed.
* **413 Payload Too Large** – Body too big.
* **415 Unsupported Media Type** – Wrong `Content-Type`.
* **418 I’m a teapot** – Easter egg (don’t use in prod 😄).
* **422 Unprocessable Entity** – Semantically invalid (FastAPI/Pydantic validation).
* **429 Too Many Requests** – Rate limit hit (include `Retry-After`).

**5xx — Server Errors**

* **500 Internal Server Error** – Generic server crash/bug.
* **501 Not Implemented** – Method/feature not supported.
* **502 Bad Gateway** – Upstream error (proxy/gateway).
* **503 Service Unavailable** – Temporarily overloaded/maintenance (use `Retry-After`).
* **504 Gateway Timeout** – Upstream didn’t respond in time.

When to use what:

* **GET** success: `200` (or `206` if partial), `304` if cached.
* **POST (create)**: `201` + `Location: /items/{id}`.
* **POST (enqueue job)**: `202` + job status URL.
* **PUT/PATCH (update)**: `200` with body or `204` without body.
* **DELETE**: `204` (or `200` with a body), `404` if not found.
* **Validation failure**: `422` (FastAPI default).
* **Auth**: `401` (no/invalid creds), `403` (not allowed).
* **Conflict**: `409` (duplicate, version mismatch).



You don’t need to define every status code. FastAPI/Uvicorn handle many for you. You only set codes when you want something different from the defaults.

What FastAPI handles automatically:

 * 200 OK on success if you just return ... (your /fft and /items do this).

 * 404 Not Found when the route doesn’t exist (e.g., POST /items/42 if not defined).

 * 405 Method Not Allowed when the path exists but the HTTP method doesn’t.

 * 422 Unprocessable Entity when request data fails validation (e.g., price:"p").

 * 500 Internal Server Error for uncaught exceptions.


## Interactive API docs page


Visit [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs). This is an interactive API docs page that FastAPI serves automatically.

It’s powered by Swagger UI and reads your app’s OpenAPI schema to render endpoints, parameters, models, and example responses.

You can “Try it out”: send real requests from the browser, see status codes, headers, and JSON.



OpenAPI is the specification/standard for describing HTTP APIs (paths, methods, params, schemas, errors). It used to be called Swagger. 

You can customize  the doc header, for example:

```python
app = FastAPI(
    title="Signal Lab API",
    description="Endpoints for FFT demos (sin-cos combos) and simple items CRUD.",
    version="0.1.0",
    contact={"name": "Boris Bolliet"},
    license_info={"name": "MIT"},
)
```


## Note on REST

There is a link between [REST](https://en.wikipedia.org/wiki/REST) and FastAPI. 

REST is an architectural style (resources + HTTP verbs + status codes + statelessness).

FastAPI is a web framework that gives you the tools to implement that style cleanly:

* Decorators for HTTP methods: @app.get/post/put/patch/delete.

* Path params & query params → map naturally to resource identifiers and filters.

* Pydantic models enforce request/response schemas (great for REST contracts).

* Status codes & headers are simple to set (status_code=201, Location, etc.).

* OpenAPI docs auto-generated so your REST contract is visible at /docs.


FastAPI encourages REST best practices but doesn’t force them—.

If you follow REST conventions (nouns for paths, proper verbs, status codes, headers), FastAPI gives you type-safe validation and live docs “for free.”

# Frontend: Next js

## Structure and Initialization


Broadly the structyre will look like:

```bash
  backend/        # your FastAPI app
    main.py
    requirements.txt
  frontend/       # your Next.js app
    package.json
    app/
```

You can just create the backend folder, move the main.py script there, and then execute: 

```bash
npx create-next-app@latest frontend --ts --eslint --tailwind
```

It scaffolds a ready-to-run Next.js project named `frontend` with TypeScript, ESLint, and Tailwind preconfigured. Breakdown:

* **`npx`**: runs the package without installing it globally (grabs the latest `create-next-app`).
* **`create-next-app@latest`**: the official Next.js project generator (using the newest version).
* **`frontend`**: the folder/name of your new app.
* **`--ts`**: sets up **TypeScript** (`tsconfig.json`, `next-env.d.ts`, `.tsx` pages/components).
* **`--eslint`**: adds **ESLint** with Next.js rules (includes `next/core-web-vitals`) and a `.eslintrc`. ESLint is a static analysis (linting) tool for JavaScript/TypeScript. It scans your code without running it and flags problems—bugs, anti-patterns, style issues, and security foot-guns—based on a set of rules.
* **`--tailwind`**: installs and wires **Tailwind CSS** (`tailwind.config.js`, `postcss.config.js`, adds `@tailwind` directives to `globals.css`).

We get:

* A Next.js app (App Router by default) with sensible defaults.
* Dependencies installed (`next`, `react`, `react-dom`, `tailwindcss`, `postcss`, `autoprefixer`, ESLint plugins).
* Git initialized (unless disabled), `.gitignore`, `README.md`, and npm scripts in `package.json`.

After that we get: 

```bash
% tree -L 2                                                   
.
├── backend
│   └── main.py
└── frontend
    ├── README.md
    ├── eslint.config.mjs
    ├── next-env.d.ts
    ├── next.config.ts
    ├── node_modules
    ├── package-lock.json
    ├── package.json
    ├── postcss.config.mjs
    ├── public
    ├── src
    └── tsconfig.json
```


## Run it

How to run it:

```bash
cd frontend
npm run dev
```

This starts the dev server on **[http://localhost:3000](http://localhost:3000)**.


It tells us: 

`To get started, edit the page.tsx file.`

This file is in `frontend/src/app/page.tsx`. You can play around modifying the text.


## Bringing endpoints in

Now we want to call our backend from the frontend. 

We add `CORS`:

```python
# fastapi main.py
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000"],  # your Next.js dev URL
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
```

We add an environment variable to link the ports. For that we create: `frontend/.env.local` and write:

```
NEXT_PUBLIC_API_URL=http://127.0.0.1:8000
```

Then, in `frontend/src/app/page.tsx`, we paste:

```tsx
// app/page.tsx
"use client";
import { useState } from "react";

type FftResponse = {
  k: number; n: number; L: number;
  bin_index: number;
  ideal_bin_float: number;
  bin_offset_from_integer: number;
  Y_m_real: number; Y_m_imag: number; Y_m_abs: number;
  Y_m_norm_real: number; Y_m_norm_imag: number; Y_m_norm_abs: number;
  note: string;
};

export default function Page() {
  const [n, setN] = useState(4096);
  const [L, setL] = useState(2 * Math.PI);
  const [k, setK] = useState(14);
  const [data, setData] = useState<FftResponse | null>(null);
  const [loading, setLoading] = useState(false);
  const [err, setErr] = useState<string | null>(null);

  const run = async () => {
    try {
      setLoading(true); setErr(null);
      const url = new URL(`${process.env.NEXT_PUBLIC_API_URL}/fft`);
      url.search = new URLSearchParams({
        n: String(n),
        L: String(L),
        k: String(k),
      }).toString();

      const res = await fetch(url.toString());
      if (!res.ok) throw new Error(await res.text());
      const j = (await res.json()) as FftResponse;
      setData(j);
    } catch (e: any) {
      setErr(e.message ?? "Request failed");
    } finally {
      setLoading(false);
    }
  };

  return (
    <main className="p-6 space-y-4">
      <h1 className="text-2xl font-bold">Signal Lab UI</h1>

      <div className="grid grid-cols-3 gap-3 max-w-xl">
        <label className="flex flex-col">
          <span>N</span>
          <input className="border rounded p-2" type="number" value={n}
                 onChange={e => setN(parseInt(e.target.value || "0"))}/>
        </label>
        <label className="flex flex-col">
          <span>L</span>
          <input className="border rounded p-2" type="number" step="any" value={L}
                 onChange={e => setL(parseFloat(e.target.value || "0"))}/>
        </label>
        <label className="flex flex-col">
          <span>k</span>
          <input className="border rounded p-2" type="number" value={k}
                 onChange={e => setK(parseInt(e.target.value || "0"))}/>
        </label>
      </div>

      <button
        onClick={run}
        disabled={loading}
        className="px-4 py-2 rounded-xl shadow bg-black text-white disabled:opacity-50"
      >
        {loading ? "Computing…" : "Compute FFT bin"}
      </button>

      {err && <p className="text-red-600">{err}</p>}

      {data && (
        <pre className="p-4 rounded-xl bg-gray-100 overflow-auto">
          {JSON.stringify(data, null, 2)}
        </pre>
      )}
    </main>
  );
}

```


